# Guided Project: Exploring eBay Car Sales Data

## Open dataset

W tym projekcie będziemy pracować z zestawem danych używanych samochodów z eBay. Celem tego projektu będzie oczyszczenie danych i analiza zawartych w nich ofert samochodów używanych.

Zaczniemy od zaimportowania potrzebnych nam bibliotek i wczytania zbioru danych.

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
url = 'http://bioinf-mw.bihz.upwr.edu.pl/resources/dataquest/autos.csv'


autos = pd.read_csv(url, encoding='Latin-1')
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [ ]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

Na podstawie dotychczasowych obserawacji, zauważamy że:

* Zbiór danych zawiera 20 kolumn
* Nazwy kolumn używają camelcase zamiast preferowanego przez Pythona snakecase, co oznacza, że nie możemy po prostu zastąpić spacji podkreśleniami.
* Niektóre kolumny mają wartości null.

## Converting column names from camelcase to snakecase

Tworzymy funkcje **cleaned**, dzięki której zamienimy system notacji na snakecase. \\
Zmieniamy nazwy kolumn oraz zamieniamy znaki specjalne na _.

In [ ]:
def cleaned(col):
    col = col.replace('yearOfRegistration','registration_year')
    col = col.replace('notRepairedDamage','unrepaired_damage')
    col = col.replace('monthOfRegistration','registration_month')
    col = col.replace('dateCreated','ad_created')
    col = re.sub( '(?<!^)(?=[A-Z])', '_', col ).lower()
    return col

Sprawdzamy, czy nazwy zmieniły się poprawnie.

In [ ]:
autos.columns = [cleaned(x) for x in autos.columns]
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_p_s', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

Sprawdzamy jak teraz wygląda nasz zbiór danych.

In [ ]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Exploring columns

In [ ]:
autos.describe(include = 'all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Na podstawie powyższej tabelki widzimy że:

* istnieją kolumny, które mają przeważnie jedną wartość i są kandydatami do usunięcia (np. nr_of_pictures),
* w tabelce występują dane liczbowe przechowywane jako tekst (np. price)

Przystępujemy do oczyszczania danych.

## Cleaning odometer column and rename to odometer_km

Zmieniamy dane w kolumnie odometer. Usuwamy wystąpienia wyrażenia "km" orax "," i przypisujemy dane do typu int. Następnie nazwe kolumny zmieniamy na odometer_km i sprawdzamy jej wartości.

In [ ]:
autos["odometer"] = (autos["odometer"]
                             .str.replace("km","")
                             .str.replace(",","")
                             .astype(int)
                             )
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)
autos["odometer_km"].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int64

Podobne działania wykonujemy dla kolumny price.

In [ ]:
autos["price"] = (autos["price"]
                 .str.replace("$", "")
                 .str.replace(",","")
                 .astype(int)
)

autos["price"].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

## Exploring odometer_km and price

Przeanalizujemy kolumny używając wartości minimalnych i maksymalnych. Naszym celem jest znalezienie wartości odstających, które moglibyśmy usunąć.

Zaczniemy od kolumny odometer_km.

In [ ]:
print(autos["odometer_km"].unique().shape)
print(autos["odometer_km"].describe())
print(autos["odometer_km"].value_counts().head(10))

(13,)
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
Name: odometer_km, dtype: int64


Istnieje 13 unikalnych wierszy danych dla kolumny odometer_km, ale nie ma w tym zbiorze tendencji do wartości nierealistycznych.

Przechodzimy więc do kolumny price.

In [ ]:
print(autos['price'].unique().shape)
print(autos['price'].describe())
print(autos['price'].value_counts().head(10))

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64
0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
Name: price, dtype: int64


W zbiorze znajduje się 1421 pozycji z ceną wynoszącą 0. Należałoby takie dane usunąć z całego zbioru. Wartość maksymalna jest również nierealistycznie wysoka, dlatego warto byłoby się przyjrzeć największym cenom w całym zbiorze w pozycji malejącej.

In [ ]:
autos['price'].value_counts().sort_index(ascending = False).head(40)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
194000      1
190000      1
180000      1
175000      1
169999      1
169000      1
163991      1
163500      1
155000      1
151990      1
145000      1
139997      1
137999      1
135000      1
130000      1
129000      1
128000      1
120000      2
119900      1
119500      1
Name: price, dtype: int64

Przeskok danych zaczyna się od wartości 350000\$ (po niej następuje odrazu wyższa wartość 999990$). Możemy w tym przypadku przyjąć dane wyższe od tej liczby za zbyt nierealistyczne i usunąć je ze zbioru danych. \\
Przy okazji pozbędziemy się cen zaczynających się od 0\$, ponieważ są nierealne.

In [ ]:
autos = autos[autos['price'].between(1, 350000)]

Sprawdzamy teraz nasz zbiór danych.

In [ ]:
autos['price'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

## Exploring columns - continued

Przechodzimy do obliczenia rozkładu wartości w kolumnach date_crawled, ad_created i last_seen w formie procentów. Kolumny te są identyfikowane przez pandas jako wartości łańcuchowe i są reprezentowane jako ciągi znaków, więc musimy przekonwertować dane na reprezentację numeryczną, abyśmy mogli zrozumieć je ilościowo. 

Najpierw sprawdzimy jak sformatowane są wartości w trzech kolumnach łańcuchowych.

In [ ]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


Pierwsze 10 znaków reprezentuje dzień. Aby zrozumieć zakres dat, możemy wyodrębnić tylko wartości daty.

In [ ]:
(autos['date_crawled']
        .str[:10]
        .value_counts(normalize = True, dropna = False)
        .sort_index()
        )

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [ ]:
(autos['ad_created']
        .str[:10]
        .value_counts(normalize = True, dropna = False)
        .sort_index()
        )

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, Length: 76, dtype: float64

In [ ]:
(autos['last_seen']
        .str[:10]
        .value_counts(normalize = True, dropna = False)
        .sort_index()
        )

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

Widzimy, że:
* Pozycje ukazane w data_crawled oraz ad_created są w podobnych ramach czasowych. Ewentualne różnice mogą wynikać z wyłączenia reklam kiedy auto zostało już sprzedane. Wówczas reklama nie pojawiała się na stronie, co możliwie nie wyłapał data_crawled. \\
* Last_seen prawidłowo pokazuje datę usunięcie reklamy z wirtyny.

Przechodzimy do kolejnej kolumny.

In [ ]:
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Jest to kolumna pokazująca date rejestracji. Max (9999) oraz min (1000) nie są realistyczne. \\
Zawężymy date rejestracji do 2016, ponieważ wtedy zostały pobrane dane i nie ma możliwości, aby przekroczyć ten okres czasowy. Sytuacja w drugą stronę jest trudniejsza, ponieważ auta mogły mieć datę rejestracji bardzo odbiegającą w przeszłość, np. podczas sprzedaży antyku. W tym projekcie zdecydujmy aby zawęzić dane do roku 1886. Jest to data powstania Patentwagen Nr. 1 konstrukcji Carla Benza, uważanego za pełnoprawny pierwszy samochód.

In [ ]:
autos = autos[autos['registration_year'].between(1886, 2016)]
autos['registration_year'].value_counts(normalize = True).head(10)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
2006    0.057197
2001    0.056468
2002    0.053255
1998    0.050620
2007    0.048778
Name: registration_year, dtype: float64

## Exploring Price by Brand

Sprawdzamy teraz marki samochodów.

In [ ]:
autos['brand'].value_counts(normalize = True)

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

Zawęzimy marki na te, które stanowią 3% wszystkich danych sprzedażowych. (W oryginalnej instrukcji podane jest 5%, dla urozmaicenia wybrałam 3%)

In [ ]:
brands = autos['brand'].value_counts(normalize = True)
common_brands = brands[brands > 0.03].index
common_brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford',
       'renault'],
      dtype='object')

Zróbmy analizę sprzedażową wybranych marek do analizy za pomocą wyliczenia średnich. Dla wszystkich marek przypisujemy .mean(), a następnie odpowiednie marki i przypisane średnie dodajemy do pustego słownika brand_mean_prices. 

In [ ]:
brand_mean_prices = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)

brand_mean_prices

{'audi': 9336,
 'bmw': 8332,
 'ford': 3749,
 'mercedes_benz': 8628,
 'opel': 2975,
 'renault': 2474,
 'volkswagen': 5402}

Nasza top. 3 to Audi, BMW oraz Mercedes Benz. Najmniej sprzedającą się marką  według tego zastanowienia to Renault.

Porównując to do obecnych staystyk, to Toyota, Skoda i Volkswagen są markami zajmujące pierwsze miejsca w opublikowanym właśnie przez SAMAR podsumowaniu roku 2021. Jak możemy zauważyć, trendy szybko się zmieniają.

In [ ]:
series = pd.Series(brand_mean_prices).sort_values(ascending = False)
pd.DataFrame(series, columns=['Średnia cena'])

,Średnia cena
audi,9336
mercedes_benz,8628
bmw,8332
volkswagen,5402
ford,3749
opel,2975
renault,2474


Naszym zadaniem jest również wyliczenie średniego przebiegu(mean mileage). Wykonujemy działania podobne jak wykonywaliścy dla kolumny price. Przypisujemy marki do brand_only, następnie ich średnie za pomocą .mean() i dodajemy je do słownika brand_mean_mileage.

In [ ]:
brand_mean_mileage = {}

for brands in common_brands:
    brand_only = autos[autos['brand'] == brands]
    mean_mileage = brand_only['odometer_km'].mean()
    brand_mean_mileage[brands] = int(mean_mileage)
brand_mean_mileage

{'audi': 129157,
 'bmw': 132572,
 'ford': 124266,
 'mercedes_benz': 130788,
 'opel': 129310,
 'renault': 128071,
 'volkswagen': 128707}

Największy przebieg z tej puli ma BMW, najmniejszy Ford. W tabelce wyglądałoby to tak:

In [ ]:
series2 = pd.Series(brand_mean_mileage).sort_values(ascending = False)
pd.DataFrame(series2, columns=['Średni przebieg'])

,Średni przebieg
bmw,132572
mercedes_benz,130788
opel,129310
audi,129157
volkswagen,128707
renault,128071
ford,124266


Możemy zrobić wspólną tabelę dla obu kolumn.

In [ ]:
mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending = False)
mean_price = pd.Series(brand_mean_prices).sort_values(ascending = False)
brand_info = pd.DataFrame(mean_mileage, columns =['Średni przebieg'])
brand_info['Średnia cena'] = mean_price
brand_info

,Średni przebieg,Średnia cena
bmw,132572,8332
mercedes_benz,130788,8628
opel,129310,2975
audi,129157,9336
volkswagen,128707,5402
renault,128071,2474
ford,124266,3749


Bazując na tej tabelce zauważamy zależność, że przeważnie droższe auta mają większe przebiegi.

Na tym etapie przestudiowaliśmy zbiór danych eBay Car Sales.